In [1]:
from download_data import get_titles_list, BASE_URL, get_project_root
import requests

# r = get_titles_list()

In [3]:
TEST_TITLE = 1
title = TEST_TITLE
versions = f"/api/versioner/v1/versions/title-{title}.json"

r = requests.get(BASE_URL + versions)

In [5]:
root = get_project_root()
title_changes_path = root / "data" / "title_changes"
if not title_changes_path.exists():
    title_changes_path.mkdir(parents=True)
with open(title_changes_path / f"title_{title}_changes.json", "w") as f:
    f.write(r.text)